# Study impact of $(M_x)_d$ cut
last edit Sep-8, 2022

## Imports and definitions

In [1]:
import sys; 
software_path = '/Users/erezcohen/Desktop/Software/'
sys.path.insert(0, software_path + '/mySoftware/Python/');
sys.path.insert(0, software_path + '/CLAS12/BAND/SIDIS_at_BAND/PythonAnalysis/AcceptanceCorrections/');
sys.path.insert(0, software_path + '/CLAS12/BAND/SIDIS_at_BAND/PythonAnalysis/python_auxiliary/');
from my_tools                     import *; 
from plot_tools                   import *;
from my_data_analysis_tools       import *;
from acceptance_correction_tools  import *;
from sidis_analysis_tools         import *;
# from event_selection_tools        import *;

In [2]:
%config InlineBackend.figure_format = 'retina'
plt.rcParams['mathtext.fontset']    = 'stix'
plt.rcParams['font.family']         = 'STIXGeneral'

# (1) Load data:
All runs of $(e,e'\pi)$ data and all runs of $(e,e'\pi n)$ - as with a small number of runs normalization is off

In [3]:
load_SIDIS_data( Nruns = -1,
                do_e_e_pi=True, 
                subdirname = "With_W0.5_cut_and_M_x_d",
                do_e_e_pi_n=False, 
                do_e_e_pi_FreeP=False,
                fdebug=1, do_all_vars=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/erezcohen/Desktop/data/BAND/SIDIS_skimming/sidisdvcs/With_W0.5_cut_and_M_x_d/skimmed_SIDIS_sidisdvcs_006429_e_piplus_selected_eepi_kinematics.csv'

# (2) Apply selection cuts not previously imposed
Except from $W$ cut

In [ ]:
e_e_pi_pass_cuts,_,_,_ = apply_further_selection_cuts_to_data(doApply_minPn_cut=True, 
                                                              fdebug=2,                                                               
                                                              NeventsMax=-1);

# (3) plot $M_x$ vs. $(M_x)_d$ 

## For the data after all cuts except the $W$ cut that was not applied in August

In [ ]:
fig = plt.figure(figsize=(16,6), tight_layout=True)
for pi_ch,pi_print,pi_label,pi_color,pi_idx in zip(pi_charge_names,pi_prints,pi_labels,pi_colors,[1,2]):

    df = e_e_pi_pass_cuts[pi_ch]
    # print(len(df))
    
    ax = fig.add_subplot(1,2,pi_idx)
    plt.hist2d( df.M_x, df.M_x_d, bins=(np.linspace(1.2,3.,100),np.linspace(2.2,4.5,100)), cmap='hot_r')

    # plt.plot( [2.5,2.5], ax.get_ylim(), '-b')
    plt.plot( ax.get_xlim(),[2.5,2.5],     '-b')
    plt.fill_between( [0,3], [2.5, 2.5] , 'k', alpha=0.1)
    
    set_axes(ax,'$M_x$ [GeV/c$^2$]',
             '$(M_x)_d$ [GeV/c$^2$]' if pi_idx==1 else '', 
             remove_ticks_y= False if pi_idx==1 else True,
             title="$(e,e'"+pi_label+")$ passed cuts",
             do_add_grid=True, do_add_legend=False)

# (4) Extract tabular data and save results to file

In [ ]:
z_bins   = np.arange(0.3,0.85,0.05)
z_widths = 0.01*np.ones(len(z_bins))
x_bins   = np.linspace(0.2,0.6,11)
x        = (x_bins[1:] + x_bins[:-1])/2
x_err    = (x_bins[1:] - x_bins[:-1])/2
print('x bins:',x_bins)
print('z bins:',z_bins)
for z_bin,z_width in zip(z_bins,z_widths):
    z_min,z_max = z_bin-z_width, z_bin+z_width
    print('%.3f < z < %.3f:'%(z_min,z_max))
    
var_label = "standing proton x"
var    = 'xB'
xlabel = "Bjorken $x$"


## (4.1) With a cut on $(M_x)_d>2.5$

In [ ]:
extract_SIDIS_ratio(df_dict  = e_e_pi_pass_cuts,
                    data_path= '/Users/erezcohen/Desktop/data/BAND/Results/ImpactOfMxcut/',
                    x_var    = var ,                           
                    x_bins   = x_bins,                           
                    z_bins   = z_bins,                           
                    z_widths = z_widths,    
                    Mx_d_min = 2.5,
                    fdebug   = 0,
                    prefix   = 'Untagged_SIDIS_ratio_',                           
                    suffix   = 'With_Mx2.5_cut')

In [ ]:
prefix = 'Untagged_SIDIS_ratio_'
suffix = 'With_Mx2.5_cut'
SIDIS_results = load_SIDIS_ratio(prefix = prefix, suffix = suffix, doPlotResults=True,
                                 data_path='/Users/erezcohen/Desktop/data/BAND/Results/ImpactOfMxcut/')
# ax = plt.gca()
# ax.set_ylim(1,2.5);

## (4.2) With no cut on $(M_x)_d$

In [ ]:
extract_SIDIS_ratio(df_dict  = e_e_pi_pass_cuts,
                    data_path= '/Users/erezcohen/Desktop/data/BAND/Results/ImpactOfMxcut/',
                    x_var    = var ,                           
                    x_bins   = x_bins,                           
                    z_bins   = z_bins,                           
                    z_widths = z_widths, 
                    W_min    = 0.5,
                    fdebug   = 0,
                    prefix   = 'Untagged_SIDIS_ratio_',                           
                    suffix   = 'With_no_Mx_cut')

In [ ]:
prefix = 'Untagged_SIDIS_ratio_'
suffix = 'With_no_Mx_cut'
SIDIS_results = load_SIDIS_ratio(prefix = prefix, suffix = suffix, doPlotResults=True,
                                 data_path='/Users/erezcohen/Desktop/data/BAND/Results/ImpactOfMxcut/')



## (4.3) Compare with and without $(M_x)_d>2.5$ GeV cut

In [ ]:
SIDIS_res = dict()
prefix = 'Untagged_SIDIS_ratio_'
for suffix in ['With_Mx2.5_cut','With_no_Mx_cut']:
    SIDIS_res[suffix] = load_SIDIS_ratio(prefix = prefix, suffix = suffix, 
                                         doPlotResults=False, 
                                         data_path='/Users/erezcohen/Desktop/data/BAND/Results/ImpactOfMxcut/')
print('Done.')

In [ ]:
z_arr,z_err_arr,R_arr,R_errup_arr,R_errdw_arr = dict(),dict(),dict(),dict(),dict()
for suffix in ['With_Mx2.5_cut','With_no_Mx_cut']:
    # print(suffix)
    results = SIDIS_res[suffix]
    z_arr[suffix],z_err_arr[suffix],R_arr[suffix],R_errup_arr[suffix],R_errdw_arr[suffix] = [],[],[],[],[]
    for key in results.keys():
        # print(key)
        z_min = float(key[4:9])
        z_max = float(key[-4:])

        # print(z_min,'<z<',z_max)
        res_x034 = results[key][np.abs(results[key]['$x_B$']-0.34)<0.02]
        # print(results[key])
        R,dR_up,dR_dw = float(res_x034['$R$']),float(res_x034['$\Delta R_{+}$']),float(res_x034['$\Delta R_{+}$'])
        # print(R,dR_up,dR_dw)
        
        R_arr[suffix].append( R )
        R_errup_arr[suffix].append( dR_up )
        R_errdw_arr[suffix].append( dR_dw )        

        z_arr[suffix].append( (z_max + z_min)/2 )
        z_err_arr[suffix].append( (z_max - z_min)/2 )
        
print('Done.')

In [ ]:
fig = plt.figure(figsize=(9,6), tight_layout=True)
ax = fig.add_subplot(1,1,1)

for suffix,sufname,sufidx in zip(['With_Mx2.5_cut','With_no_Mx_cut'],
                                 ['$(M_x)_d>2.5$ GeV/c$^2$','no $(M_x)_d$ cut'],
                                 [0,1]):
    plt.errorbar(x=np.array(z_arr[suffix])+sufidx*0.005, xerr=z_err_arr[suffix],
                 y=R_arr[suffix], yerr=(R_errup_arr[suffix],R_errdw_arr[suffix]),
                 marker='o',markeredgecolor='k',linestyle='None',
                 label=sufname, capthick=2, capsize=2)

set_axes(ax,'$z_\\pi$',
             "$\\sigma_{d(e,e'\\pi^+)}/\\sigma_{d(e,e'\\pi^-)}$ ",
             do_add_grid=True, do_add_legend=True)
ax.legend(loc='lower right',fontsize=24);

In [ ]:
z_arr['No_W_cut']